# Task

In [552]:
import pandas as pd
import json

In [553]:
df = pd.read_csv(r'C:\Users\irakl\Desktop\assignment\data.csv')

### Data type changes

In [554]:
df_json = df.copy(deep = True)
df_json['id'] = df_json['id'].astype('int')
df_json['application_date_timestamp'] =  df_json['application_date'].apply(lambda x: pd.Timestamp(x))
df_json['application_date_short_timestamp'] = df_json['application_date_timestamp'].dt.tz_localize(None).dt.strftime('%Y-%m-%d %H:%M:%S.%f').str[:-3]. \
apply (lambda x: pd.Timestamp(x))

### JSON flattening process

In [555]:
#Cast JSON data to dictionaries using json.loads() function
df_json_to_dict = df_json['contracts'].apply(lambda x: json.loads(x) if isinstance(x, str) else {})

In [556]:
#Flatten JSON using pd.json_normalize() function
df_json_normalized = df_json_to_dict.apply(lambda x: pd.json_normalize(x))

In [557]:
#Result was series of DataFrames. 
#Creation of one uniform Dataframe column from series objects is possible by stacking series 
if isinstance(df_json_normalized, pd.Series):
    # Convert the Series of DataFrames to a list of DataFrames
    df_json_normalized_list = df_json_normalized.tolist()
    #Keeping Dataframe Number by assigning index to DataFrame is essential to keep track of the user_id (df['id']) 
    df_json_normalized_list = [df.assign(list_index=i) for i, df in enumerate(df_json_normalized_list)]
    #Concatenate the list into a single DataFrame
    df_combined = pd.concat(df_json_normalized_list, ignore_index=True)
else:
    #If it's already a DataFrame, no need to concatenate
    df_combined = df_json_normalized

#Merge with the original DataFrame
#Create explicit index for merge. This index can be used as a join on parameter 
df_json = df_json.reset_index()
#Merge on indicies
df_combined_final = pd.merge(
     df_json.drop(columns=['contracts']),  
    df_combined,                      
    left_on='index',                     
    right_on='list_index',             
    how='inner'                        
)

In [558]:
#Replacing empty strings and true null values with pandas NA value
df_combined_final['contract_id'] = df_combined_final['contract_id'].apply(lambda x: pd.NA if pd.isnull(x) or x=='' else x)
df_combined_final['bank'] = df_combined_final['bank'].apply(lambda x: pd.NA if pd.isnull(x) or x=='' else x)
df_combined_final['summa'] = df_combined_final['summa'].apply(lambda x: pd.NA if pd.isnull(x) or x=='' else x)
df_combined_final['loan_summa'] = df_combined_final['loan_summa'].apply(lambda x: pd.NA if pd.isnull(x) or x=='' else x)
df_combined_final['claim_date'] = df_combined_final['claim_date'].apply(lambda x: pd.NA if pd.isnull(x) or x=='' else x)
df_combined_final['claim_id'] = df_combined_final['claim_id'].apply(lambda x: pd.NA if pd.isnull(x) or x=='' else x)
df_combined_final['contract_date'] = df_combined_final['contract_date'].apply(lambda x: pd.NA if pd.isnull(x) or x=='' else x)

## Features

#### Same steps works for all the features. Copying flattened Data, filtering and grouping it 

### Total Claims Count 280 Days

In [559]:
feature_claims_raw =  df_combined_final.copy(deep = True)

In [560]:
feature_claims_filtered = feature_claims_raw.loc[(pd.to_datetime(feature_claims_raw['claim_date'], format= '%d.%m.%Y') >=  pd.Timestamp.now()- pd.Timedelta(days=280))  & (feature_claims_raw['claim_date'].notna()) ]

In [561]:
feature_claims_grouped = feature_claims_filtered[['id','claim_date']].groupby('id').count()
feature_claims_grouped.reset_index(inplace=True)

### Sum of Exposure of loans WO TBC

In [562]:
feature_exposure_raw =  df_combined_final.copy(deep = True)

In [563]:
feature_exposure_filtered = feature_exposure_raw.loc[(~feature_exposure_raw['bank'].isin(['LIZ', 'LOM', 'MKO', 'SUG', pd.NA]))\
&  (feature_exposure_raw['contract_date'].notna()) ]

In [564]:
feature_exposure_grouped =feature_exposure_filtered[['id', 'loan_summa']].groupby('id').sum()
feature_exposure_grouped.reset_index(inplace=True)

### Number of Days Since Last Loan

In [565]:
feature_last_loan_raw =  df_combined_final.copy(deep = True)

In [566]:
feature_last_loan_filtered = feature_last_loan_raw.loc[feature_last_loan_raw['summa'].notna()]

In [567]:
feature_last_loan_grouped = feature_last_loan_filtered[['id','application_date_short_timestamp', 'contract_date']].groupby(['id','application_date_short_timestamp']). \
agg({
    'contract_date': 'max'})
feature_last_loan_grouped.reset_index(inplace = True)

In [568]:
feature_last_loan_grouped['days_since_last_loan']=(feature_last_loan_grouped['application_date_short_timestamp'] - pd.to_datetime(feature_last_loan_grouped['contract_date'], format= '%d.%m.%Y')).dt.days

## Data Merging

In [569]:
df_initial = df_json.copy(deep = True)

In [570]:
#Initial and claims feature merge 
df_result = pd.merge(
    df_initial.drop(columns=['contracts']), 
    feature_claims_grouped,                     
    left_on='id',                  
    right_on='id',    
    how='left'    
)

In [571]:
#Result and exposure feature merge
df_result = pd.merge(
    df_result, 
    feature_exposure_grouped,                     
    left_on='id',                  
    right_on='id',    
    how='left'    
)

In [572]:
#Result and last loan feature merge 
df_result = pd.merge(
    df_result, 
    feature_last_loan_grouped,                     
    left_on='id',                  
    right_on='id',    
    how='left'    
)

In [573]:
#Cleaning up all unnecessary columns and renaming them 
df_result = df_result.drop(['index', 'application_date', 'application_date_short_timestamp_y', 'contract_date', 'application_date_timestamp'],axis =1 )
df_result = df_result.rename(columns={
    'application_date_short_timestamp_x': 'application_date',
    'claim_date': 'number_of_claims',
    'loan_summa': 'total_exposure'
})

#### Applying Feature Missing Value Rules

In [574]:
df_result['number_of_claims'].fillna(-3, inplace = True)
df_result['number_of_claims'] = df_result['number_of_claims'].astype('int')

In [575]:
df_result['total_exposure'] = df_result.apply(lambda row: row['total_exposure'] 
        if not pd.isna(row['total_exposure']) else -3 if row['number_of_claims'] ==-3 else -100 if (pd.isna(row['total_exposure'])) & (not pd.isna(row['days_since_last_loan'])) else  -1 if pd.isna(row['total_exposure'])   else -1000, axis=1)

In [582]:
df_result['days_since_last_loan'] = df_result.apply(lambda row: row['days_since_last_loan'] 
        if not pd.isna(row['days_since_last_loan']) else -3 if row['number_of_claims'] ==-3 else -1, axis=1).astype('int')

In [586]:
#Export to CSV
df_result.to_csv('contract_features.csv', index = False, encoding='utf-8')